In [1]:
import numpy as np
import pandas as pd

# import libraries for google sheet connection
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [2]:
# define function to set up g-sheet connection
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data

# set scope
scopes = ['https://www.googleapis.com/auth/spreadsheets']

In [3]:
# define g-sheet path
gsheet_id = '1pXbqPQyELcgPeTDS-Et6rKlZ5zdoa4IjjunNWMbaQIg'
tab_name = 'Sheet1'

# save results in a dataframe
data = pull_sheet_data(scopes, gsheet_id, tab_name)
df = pd.DataFrame(data[1:], columns=data[0])

df

COMPLETE: Data copied


,Product,Product Category,Sales,Damages,Damage Rate
0,,Table,578,5,0.9%
1,,Chair,123,21,17.1%
2,,Bed,309,24,7.9%
3,,Dining Set,332,39,11.8%
4,,Mirror,380,9,2.4%
5,,Vanity,219,12,5.6%
6,,Dresser,15,38,257.1%
7,,Night stand,956,12,1.2%
